In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm import tqdm

# === CONFIG ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
print(f"💻 Using device: {device}")

PATCH_SIZE = 64
NUM_FRAMES = 10
NUM_BINS = 12

💻 Using device: cuda


In [2]:
# === GROUND TRUTH HISTOGRAM (Y_ang and Y_bkg) ===
def compute_ground_truth_histogram(optical_flows, num_bins=12, threshold=0.1):
    histogram = np.zeros(num_bins + 1)  # 12 bins for angles + 1 for no motion
    for flow in optical_flows:
        flow_x = flow[..., 0]
        flow_y = flow[..., 1]
        mag, ang = cv2.cartToPolar(flow_x.astype(np.float32), flow_y.astype(np.float32))
        hist, _ = np.histogram(ang, bins=num_bins, range=(0, 2 * np.pi))
        no_motion_count = np.sum(mag < threshold)
        histogram[:-1] += hist
        histogram[-1] += no_motion_count
    total = np.sum(histogram)
    return histogram / total if total > 0 else histogram

In [3]:
def collect_valid_volume_paths(base_dir):
    volume_paths = []
    for subdir in sorted(os.listdir(base_dir)):
        sub_path = os.path.join(base_dir, subdir)
        video_path = os.path.join(sub_path, "video_volumes")
        flow_path = os.path.join(sub_path, "optical_flows")
        if not os.path.isdir(video_path) or not os.path.isdir(flow_path):
            print(f"⚠️ Skipping {subdir}: Missing required folders.")
            continue
        print(f"⏳ Collecting volume paths in {subdir}...")
        for vol_folder in sorted(os.listdir(video_path)):
            vol_full_path = os.path.join(video_path, vol_folder)
            flow_folder_path = os.path.join(flow_path, vol_folder)
            if os.path.isdir(vol_full_path) and os.path.isdir(flow_folder_path):
                volume_paths.append((vol_full_path, flow_folder_path))
    print(f"✅ Total volumes to load: {len(volume_paths)}")
    return volume_paths

In [4]:
def load_yang_data(base_dir):
    from tqdm import tqdm
    volume_paths = collect_valid_volume_paths(base_dir)
    video_volumes, y_ang = [], []

    for vol_path, flow_path in tqdm(volume_paths, desc="📥 Loading Y_ang"):
        frames, flows = [], []
        for i in range(NUM_FRAMES):
            frame_path = os.path.join(vol_path, f"frame_{i:04d}.png")
            img = cv2.imread(frame_path)
            if img is None:
                print(f"⚠️ Missing frame {i} in {vol_path}")
                frames = []
                break
            frames.append(cv2.resize(img, (PATCH_SIZE, PATCH_SIZE)))
        if len(frames) != NUM_FRAMES:
            continue
        video_volumes.append(np.stack(frames, axis=0))

        for i in range(NUM_FRAMES - 1):
            flow_file = os.path.join(flow_path, f"flow_raw_{i:04d}.npy")
            if not os.path.exists(flow_file):
                print(f"⚠️ Missing flow file: {flow_file}")
                flows = []
                break
            flows.append(cv2.resize(np.load(flow_file), (PATCH_SIZE, PATCH_SIZE)))
        if len(flows) != NUM_FRAMES - 1:
            continue

        hist = compute_ground_truth_histogram(flows, num_bins=NUM_BINS)
        y_ang.append(hist[:-1])

    if not video_volumes or not y_ang:
        raise ValueError("🚫 No valid Y_ang data found.")

    X = np.stack(video_volumes).transpose(0, 4, 1, 2, 3)
    Y_ang = torch.tensor(np.stack(y_ang), dtype=torch.float32)
    Y_ang = Y_ang / (Y_ang.sum(dim=-1, keepdim=True) + 1e-8)
    print(f"✅ Loaded {len(X)} volumes for Y_ang, shape: {X.shape}")
    return torch.tensor(X, dtype=torch.float32), Y_ang

In [5]:
def load_ybkg_data(base_dir):
    from tqdm import tqdm
    volume_paths = collect_valid_volume_paths(base_dir)
    video_volumes, y_bkg = [], []

    for vol_path, flow_path in tqdm(volume_paths, desc="📥 Loading Y_bkg"):
        frames, flows = [], []
        for i in range(NUM_FRAMES):
            frame_path = os.path.join(vol_path, f"frame_{i:04d}.png")
            img = cv2.imread(frame_path)
            if img is None:
                print(f"⚠️ Missing frame {i} in {vol_path}")
                frames = []
                break
            frames.append(cv2.resize(img, (PATCH_SIZE, PATCH_SIZE)))
        if len(frames) != NUM_FRAMES:
            continue
        video_volumes.append(np.stack(frames, axis=0))

        for i in range(NUM_FRAMES - 1):
            flow_file = os.path.join(flow_path, f"flow_raw_{i:04d}.npy")
            if not os.path.exists(flow_file):
                print(f"⚠️ Missing flow file: {flow_file}")
                flows = []
                break
            flows.append(cv2.resize(np.load(flow_file), (PATCH_SIZE, PATCH_SIZE)))
        if len(flows) != NUM_FRAMES - 1:
            continue

        hist = compute_ground_truth_histogram(flows, num_bins=NUM_BINS)
        y_bkg.append(hist[-1])  # last bin for stationary

    if not video_volumes or not y_bkg:
        raise ValueError("🚫 No valid Y_bkg data found.")

    X = np.stack(video_volumes).transpose(0, 4, 1, 2, 3)
    Y_bkg = torch.tensor(np.stack(y_bkg), dtype=torch.float32).unsqueeze(1)
    Y_bkg = (Y_bkg > 0.5).float()
    print(f"✅ Loaded {len(X)} volumes for Y_bkg, shape: {X.shape}")
    return torch.tensor(X, dtype=torch.float32), Y_bkg

In [6]:
# === MODEL DEFINITIONS ===

class YangModel(nn.Module):
    def __init__(self):
        super(YangModel, self).__init__()
        self.conv1 = nn.Conv3d(3, 32, kernel_size=(5, 5, 5), padding=2)
        self.bn1 = nn.BatchNorm3d(32)
        self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm3d(64)
        self.pool2 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        self.conv3 = nn.Conv3d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm3d(128)
        self.pool3 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        self.fc1 = nn.Linear(128 * (PATCH_SIZE // 8) * (PATCH_SIZE // 8) * NUM_FRAMES, 256)
        self.fc_bn = nn.BatchNorm1d(256)
        self.fc_relu = nn.ReLU()
        self.fc2 = nn.Linear(256, NUM_BINS)

    def forward(self, x, extract_features=False):
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))
        x = torch.flatten(x, 1)
        x = self.fc_relu(self.fc_bn(self.fc1(x)))
        if extract_features:
            return x
        
        logits = self.fc2(x)
        probs = torch.softmax(logits, dim=-1)
        log_probs = torch.log(probs + 1e-8)  # Avoid log(0) for stability
        return log_probs

class YBkgModel(YangModel):
    def __init__(self):
        super(YBkgModel, self).__init__()
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x, extract_features=False):
        x = super().forward(x, extract_features=True)
        return torch.sigmoid(self.fc2(x)) if not extract_features else x

In [7]:
# === TRAINING LOOP ===
def train_model(model, train_loader, loss_fn, optimizer, epochs=30):
    model.train()
    for epoch in tqdm(range(epochs), desc="Training"):
        total_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            output = model(xb)
            loss = loss_fn(output, yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}: Loss = {total_loss / len(train_loader):.4f}")

In [ ]:
base_dir = "C:\\Users\\hci\\Desktop\\data_new1"
# Y_ang data only
video_volumes, y_ang = load_yang_data(base_dir)

⏳ Collecting volume paths in V1...
⏳ Collecting volume paths in V10...
⏳ Collecting volume paths in V11...
⏳ Collecting volume paths in V12...
⏳ Collecting volume paths in V13...
⏳ Collecting volume paths in V14...
⏳ Collecting volume paths in V15...
⏳ Collecting volume paths in V16...
⏳ Collecting volume paths in V17...
⏳ Collecting volume paths in V18...
⏳ Collecting volume paths in V19...
⏳ Collecting volume paths in V2...
⏳ Collecting volume paths in V20...
⏳ Collecting volume paths in V21...
⏳ Collecting volume paths in V3...
⏳ Collecting volume paths in V4...
⏳ Collecting volume paths in V5...
⏳ Collecting volume paths in V6...
⏳ Collecting volume paths in V7...
⏳ Collecting volume paths in V8...
⏳ Collecting volume paths in V9...
✅ Total volumes to load: 209848


📥 Loading Y_ang:   4%|██▎                                                     | 8539/209848 [12:37<4:30:49, 12.39it/s]

In [ ]:
# Split dataset
dataset_ang = TensorDataset(video_volumes, y_ang)
#dataset_bkg = TensorDataset(video_volumes, y_bkg)

train_size = int(0.9 * len(dataset_ang))
val_size = len(dataset_ang) - train_size

train_ang, val_ang = random_split(dataset_ang, [train_size, val_size])
#train_bkg, val_bkg = random_split(dataset_bkg, [train_size, val_size])

train_loader_ang = DataLoader(train_ang, batch_size=20, shuffle=True, drop_last=True)
#train_loader_bkg = DataLoader(train_bkg, batch_size=20, shuffle=True, drop_last=True)

In [8]:
# === TRAIN Y_ang ===
yang_model = YangModel().to(device)
opt_ang = optim.AdamW(yang_model.parameters(), lr=1e-3)
train_model(yang_model, train_loader_ang, nn.KLDivLoss(reduction="batchmean"), opt_ang)
torch.save(yang_model.state_dict(), "yang_model.pth")
print("✅ Y_ang model saved.")

Training:   3%|██▎                                                                   | 1/30 [10:43<5:11:04, 643.60s/it]

Epoch 1: Loss = 0.0346


Training:   7%|████▋                                                                 | 2/30 [21:32<5:01:55, 646.97s/it]

Epoch 2: Loss = 0.0252


Training:  10%|███████                                                               | 3/30 [32:26<4:52:34, 650.17s/it]

...

Training:  97%|██████████████████████████████████████████████████████████████████▋  | 29/30 [5:27:53<11:25, 685.66s/it]

Epoch 29: Loss = 0.0065


Training: 100%|█████████████████████████████████████████████████████████████████████| 30/30 [5:39:19<00:00, 678.66s/it]

Epoch 30: Loss = 0.0065


✅ Y_ang model saved.


In [9]:
# === TRAIN Y_bkg ===
ybkg_model = YBkgModel().to(device)
opt_bkg = optim.AdamW(ybkg_model.parameters(), lr=1e-4)
train_model(ybkg_model, train_loader_bkg, nn.BCELoss(), opt_bkg)
torch.save(ybkg_model.state_dict(), "ybkg_model.pth")
print("✅ Y_bkg model saved.")

Training:   3%|██▎                                                                   | 1/30 [11:31<5:34:18, 691.67s/it]

Epoch 1: Loss = 0.0098


Training:   7%|████▋                                                                 | 2/30 [22:53<5:20:09, 686.05s/it]

Epoch 2: Loss = 0.0000


Training:  10%|███████                                                               | 3/30 [34:22<5:09:14, 687.19s/it]

...

Epoch 28: Loss = 0.0000


Training:  97%|██████████████████████████████████████████████████████████████████▋  | 29/30 [5:32:24<11:27, 687.59s/it]

Epoch 29: Loss = 0.0000


Training: 100%|█████████████████████████████████████████████████████████████████████| 30/30 [5:43:51<00:00, 687.72s/it]

Epoch 30: Loss = 0.0000
✅ Y_bkg model saved.


In [ ]:
# ✅ Load trained models
model_ang = YangModel().to(device)
#model_bkg = YBkgModel().to(device)
model_ang.load_state_dict(torch.load("yang_model.pth"))
#model_bkg.load_state_dict(torch.load("ybkg_model.pth"))

In [ ]:
# ✅ Extract feature vectors
def extract_feature_vectors(model, dataloader, save_name):
    model.eval()
    features = []

    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            feature_vecs = model(inputs, extract_features=True)
            features.append(feature_vecs.cpu().numpy())

    features = np.vstack(features)
    np.save(f"{save_name}.npy", features)
    pd.DataFrame(features).to_csv(f"{save_name}.csv", index=False)

    print(f"✅ Feature vectors saved: {save_name}")

In [ ]:
# ✅ Save extracted feature vectors
extract_feature_vectors(model_ang, train_loader_ang, "y_ang_features_updated_test")
#extract_feature_vectors(model_bkg, train_loader_bkg, "y_bkg_features_updated_test")